In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib, pymysql, calendar, time, json
from urllib.request import urlopen
from datetime import datetime
from threading import Timer

In [2]:
class DBUpdater:
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
        self.conn = pymysql.connect(host='localhost', user='root',
                                    password='1111', db='INVESTAR', charset='utf8')

        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS company_info (
                code VARCHAR(20),
                company VARCHAR(40),
                last_update DATE,
                PRIMARY KEY (code))
            """
            curs.execute(sql)
            sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
            curs.execute(sql)
        self.conn.commit()
        self.codes = dict()

    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
        self.conn.close()

    def read_krx_code(self):
        """KRX로부터 상장기업 목록 파일을 읽어와서 데이터프레임으로 반환"""
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=' \
              'download&searchType=13'
        krx = pd.read_html(url, header=0)[0]
        krx = krx[['종목코드', '회사명']]
        krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
        krx.code = krx.code.map('{:06d}'.format)
        return krx

    def update_comp_info(self):
        """종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에 저장"""
        
        # DB 업데이트하다 중지되면 중지된 code넣어서 쓰기!
        
        # code = 222800
        
        sql = f"SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]

        with self.conn.cursor() as curs:
            sql = f"SELECT max(last_update) FROM company_info"
            curs.execute(sql)
            rs = curs.fetchone()
            today = datetime.today().strftime('%Y-%m-%d')
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
                krx = self.read_krx_code()
                for idx in range(len(krx)):
                    code = krx.code.values[idx]
                    company = krx.company.values[idx]
                    sql = f"REPLACE INTO company_info (code, company, last" \
                          f"_update) VALUES ('{code}', '{company}', '{today}')"
                    curs.execute(sql)
                    self.codes[code] = company
                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(f"[{tmnow}] #{idx + 1:04d} REPLACE INTO company_info " \
                          f"VALUES ({code}, {company}, {today})")
                self.conn.commit()
                print('')

    def read_naver(self, code, company, pages_to_fetch):
        """네이버에서 주식 시세를 읽어서 데이터프레임으로 반환"""
        try:
            url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
            with urlopen(url) as doc:
                if doc is None:
                    return None
                html = BeautifulSoup(doc, "lxml")
                pgrr = html.find("td", class_="pgRR")
                if pgrr is None:
                    return None
                s = str(pgrr.a["href"]).split('=')
                lastpage = s[-1]
            df = pd.DataFrame()
            pages = min(int(lastpage), pages_to_fetch)
            for page in range(1, pages + 1):
                pg_url = '{}&page={}'.format(url, page)
                df = df.append(pd.read_html(pg_url, header=0)[0])
                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.
                      format(tmnow, company, code, page, pages), end="\r")
            df = df.rename(columns={'날짜': 'date', '종가': 'close', '전일비': 'diff'
                , '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})
            df['date'] = df['date'].replace('.', '-')
            df = df.dropna()
            df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close',
                                                                         'diff', 'open', 'high', 'low',
                                                                         'volume']].astype(int)
            df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]
        except Exception as e:
            print('Exception occured :', str(e))
            return None
        return df

    def replace_into_db(self, df, num, code, company):
        """네이버에서 읽어온 주식 시세를 DB에 REPLACE"""
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = f"REPLACE INTO daily_price VALUES ('{code}', " \
                      f"'{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, " \
                      f"{r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.commit()
            print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_' \
                  'price [OK]'.format(datetime.now().strftime('%Y-%m-%d' \
                                                              ' %H:%M'), num + 1, company, code, len(df)))

    def update_daily_price(self, pages_to_fetch):
        """KRX 상장법인의 주식 시세를 네이버로부터 읽어서 DB에 업데이트"""
        for idx, code in enumerate(self.codes):
            df = self.read_naver(code, self.codes[code], pages_to_fetch)
            if df is None:
                continue
            self.replace_into_db(df, idx, code, self.codes[code])

    def execute_daily(self):
        """실행 즉시 및 매일 오후 다섯시에 daily_price 테이블 업데이트"""
        self.update_comp_info()

        try:
            with open('config.json', 'r') as in_file:
                config = json.load(in_file)
                pages_to_fetch = config['pages_to_fetch']
        except FileNotFoundError:
            with open('config.json', 'w') as out_file:
                pages_to_fetch = 110
                config = {'pages_to_fetch': 1}
                json.dump(config, out_file)
        self.update_daily_price(pages_to_fetch)

        tmnow = datetime.now()
        lastday = calendar.monthrange(tmnow.year, tmnow.month)[1]
        if tmnow.month == 12 and tmnow.day == lastday:
            tmnext = tmnow.replace(year=tmnow.year + 1, month=1, day=1,
                                   hour=17, minute=0, second=0)
        elif tmnow.day == lastday:
            tmnext = tmnow.replace(month=tmnow.month + 1, day=1, hour=17,
                                   minute=0, second=0)
        else:
            tmnext = tmnow.replace(day=tmnow.day + 1, hour=17, minute=0,
                                   second=0)
        tmdiff = tmnext - tmnow
        secs = tmdiff.seconds
        t = Timer(secs, self.execute_daily)
        print("Waiting for next update ({}) ... ".format(tmnext.strftime
                                                         ('%Y-%m-%d %H:%M')))
        t.start()


if __name__ == '__main__':
    dbu = DBUpdater()
    dbu.execute_daily()

C:\Anaconda\lib\site-packages\pymysql\cursors.py:170: Warning: (1050, "Table 'company_info' already exists")
  result = self._query(query)
C:\Anaconda\lib\site-packages\pymysql\cursors.py:170: Warning: (1681, 'Integer display width is deprecated and will be removed in a future release.')
  result = self._query(query)
C:\Anaconda\lib\site-packages\pymysql\cursors.py:170: Warning: (1050, "Table 'daily_price' already exists")
  result = self._query(query)


[2020-10-05 02:02] #0001 REPLACE INTO company_info VALUES (155660, DSR, 2020-10-05)
[2020-10-05 02:02] #0002 REPLACE INTO company_info VALUES (001250, GS글로벌, 2020-10-05)
[2020-10-05 02:02] #0003 REPLACE INTO company_info VALUES (294870, HDC현대산업개발, 2020-10-05)
[2020-10-05 02:02] #0004 REPLACE INTO company_info VALUES (011070, LG이노텍, 2020-10-05)
[2020-10-05 02:02] #0005 REPLACE INTO company_info VALUES (066570, LG전자, 2020-10-05)
[2020-10-05 02:02] #0006 REPLACE INTO company_info VALUES (037560, LG헬로비전, 2020-10-05)
[2020-10-05 02:02] #0007 REPLACE INTO company_info VALUES (010060, OCI, 2020-10-05)
[2020-10-05 02:02] #0008 REPLACE INTO company_info VALUES (100840, S&TC, 2020-10-05)
[2020-10-05 02:02] #0009 REPLACE INTO company_info VALUES (064960, S&T모티브, 2020-10-05)
[2020-10-05 02:02] #0010 REPLACE INTO company_info VALUES (096770, SK이노베이션, 2020-10-05)
[2020-10-05 02:02] #0011 REPLACE INTO company_info VALUES (011810, STX, 2020-10-05)
[2020-10-05 02:02] #0012 REPLACE INTO company_info VAL

[2020-10-05 02:02] #0224 REPLACE INTO company_info VALUES (094970, 제이엠티, 2020-10-05)
[2020-10-05 02:02] #0225 REPLACE INTO company_info VALUES (058420, 제이웨이, 2020-10-05)
[2020-10-05 02:02] #0226 REPLACE INTO company_info VALUES (023440, 제일제강, 2020-10-05)
[2020-10-05 02:02] #0227 REPLACE INTO company_info VALUES (080220, 제주반도체, 2020-10-05)
[2020-10-05 02:02] #0228 REPLACE INTO company_info VALUES (082270, 젬백스, 2020-10-05)
[2020-10-05 02:02] #0229 REPLACE INTO company_info VALUES (034940, 조아제약, 2020-10-05)
[2020-10-05 02:02] #0230 REPLACE INTO company_info VALUES (036930, 주성엔지니어링, 2020-10-05)
[2020-10-05 02:02] #0231 REPLACE INTO company_info VALUES (119850, 지엔씨에너지, 2020-10-05)
[2020-10-05 02:02] #0232 REPLACE INTO company_info VALUES (065060, 지엔코, 2020-10-05)
[2020-10-05 02:02] #0233 REPLACE INTO company_info VALUES (071850, 캐스텍코리아, 2020-10-05)
[2020-10-05 02:02] #0234 REPLACE INTO company_info VALUES (342550, 케이비제20호스팩, 2020-10-05)
[2020-10-05 02:02] #0235 REPLACE INTO company_info VAL

[2020-10-05 02:02] #0513 REPLACE INTO company_info VALUES (114570, 지스마트글로벌, 2020-10-05)
[2020-10-05 02:02] #0514 REPLACE INTO company_info VALUES (094850, 참좋은여행, 2020-10-05)
[2020-10-05 02:02] #0515 REPLACE INTO company_info VALUES (278280, 천보, 2020-10-05)
[2020-10-05 02:02] #0516 REPLACE INTO company_info VALUES (047820, 초록뱀, 2020-10-05)
[2020-10-05 02:02] #0517 REPLACE INTO company_info VALUES (078340, 컴투스, 2020-10-05)
[2020-10-05 02:02] #0518 REPLACE INTO company_info VALUES (330990, 케이비제19호스팩, 2020-10-05)
[2020-10-05 02:02] #0519 REPLACE INTO company_info VALUES (025880, 케이씨피드, 2020-10-05)
[2020-10-05 02:02] #0520 REPLACE INTO company_info VALUES (105330, 케이엔더블유, 2020-10-05)
[2020-10-05 02:02] #0521 REPLACE INTO company_info VALUES (033110, 코너스톤네트웍스, 2020-10-05)
[2020-10-05 02:02] #0522 REPLACE INTO company_info VALUES (047770, 코데즈컴바인, 2020-10-05)
[2020-10-05 02:02] #0523 REPLACE INTO company_info VALUES (049430, 코메론, 2020-10-05)
[2020-10-05 02:02] #0524 REPLACE INTO company_info V

[2020-10-05 02:02] #0830 REPLACE INTO company_info VALUES (054210, 이랜텍, 2020-10-05)
[2020-10-05 02:02] #0831 REPLACE INTO company_info VALUES (115610, 이미지스, 2020-10-05)
[2020-10-05 02:02] #0832 REPLACE INTO company_info VALUES (086890, 이수앱지스, 2020-10-05)
[2020-10-05 02:02] #0833 REPLACE INTO company_info VALUES (033230, 인성정보, 2020-10-05)
[2020-10-05 02:02] #0834 REPLACE INTO company_info VALUES (049070, 인탑스, 2020-10-05)
[2020-10-05 02:02] #0835 REPLACE INTO company_info VALUES (035080, 인터파크, 2020-10-05)
[2020-10-05 02:02] #0836 REPLACE INTO company_info VALUES (049630, 재영솔루텍, 2020-10-05)
[2020-10-05 02:02] #0837 REPLACE INTO company_info VALUES (089790, 제이티, 2020-10-05)
[2020-10-05 02:02] #0838 REPLACE INTO company_info VALUES (043610, 지니뮤직, 2020-10-05)
[2020-10-05 02:02] #0839 REPLACE INTO company_info VALUES (219750, 지티지웰니스, 2020-10-05)
[2020-10-05 02:02] #0840 REPLACE INTO company_info VALUES (086060, 진바이오텍, 2020-10-05)
[2020-10-05 02:02] #0841 REPLACE INTO company_info VALUES (0368

...

[2020-10-05 02:09] #1914 로보로보 (215100) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #1915 메가스터디교육 (215200) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #1916 우리산업 (215360) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #1917 우정바이오 (215380) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #1918 토박스코리아 (215480) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #1919 크로넥스 (215570) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #1920 신라젠 (215600) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #1921 이노인스트루먼트 (215790) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #1922 인크로스 (216050) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #1923 제테마 (216080) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #1924 원텍 (216280) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #1925 바이오코아 (216400) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #1

[2020-10-05 02:09] #2014 녹십자웰빙 (234690) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #2015 자이글 (234920) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #2016 메드팩토 (235980) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #2017 씨알푸드 (236030) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:09] #2018 슈프리마 (236200) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2019 메디젠휴먼케어 (236340) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2020 에스티팜 (237690) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2021 피앤씨테크 (237750) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2022 플레이디 (237820) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2023 클리오 (237880) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2024 앤디포스 (238090) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2025 얼라인드 (238120) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2026 

[2020-10-05 02:10] #2114 AP시스템 (265520) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2115 영화테크 (265560) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2116 뿌리깊은나무들 (266170) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2117 바이오인프라생명과학 (266470) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2118 파워풀엑스 (266870) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2119 명진홀딩스 (267060) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2120 현대중공업지주 (267250) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2121 현대일렉트릭 (267260) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2122 현대건설기계 (267270) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2123 경동도시가스 (267290) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2124 나인테크 (267320) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2125 배럴 (267790) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-

[2020-10-05 02:10] #2215 IBKS제10호스팩 (299170) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2216 지앤이헬스케어 (299480) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2217 셀리드 (299660) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2218 에스엠비나 (299670) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2219 위지윅스튜디오 (299900) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2220 베스파 (299910) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2221 플리토 (300080) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2222 라온피플 (300120) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2223 한일시멘트 (300720) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2224 이노메트리 (302430) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2225 리메드 (302550) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:10] #2226 더콘텐츠온 (302920) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:1

[2020-10-05 02:11] #2316 두산퓨얼셀 (336260) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:11] #2317 두산솔루스 (336370) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:11] #2318 대신밸런스제8호스팩 (336570) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:11] #2319 SK5호스팩 (337450) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:11] #2320 유엑스엔 (337840) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:11] #2321 브랜드엑스코퍼레이션 (337930) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:11] #2322 NH프라임리츠 (338100) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:11] #2323 엔에이치스팩15 (339950) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:11] #2324 하이제5호스팩 (340120) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:11] #2325 SK6호스팩 (340350) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:11] #2326 유안타제6호스팩 (340360) : 10 rows > REPLACE INTO daily_price [OK]
[2020-10-05 02:11] #2327 한화플러스제1호스팩 (340440) : 10 rows > REPLACE INTO daily_p